<a href="https://colab.research.google.com/github/manushi0304/Diabetic_Retinopathy/blob/main/Convertedmodels2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from google.colab import drive

# Mount Google Drive
drive.mount("/content/drive", force_remount=True)

# Load the Keras model
model = tf.keras.models.load_model('/content/drive/MyDrive/DiabeticProject/saved_models/InceptionV3_single/latest_model.keras')  # or 'your_model.h5'

# ---------------------
# 1. Float32 TFLite (no quantization)
# ---------------------
converter_fp32 = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model_fp32 = converter_fp32.convert()

with open('InceptionV3_model_fp32.tflite', 'wb') as f:
    f.write(tflite_model_fp32)

# ---------------------
# 2. Float16 Quantization
# ---------------------
converter_fp16 = tf.lite.TFLiteConverter.from_keras_model(model)
converter_fp16.optimizations = [tf.lite.Optimize.DEFAULT]
converter_fp16.target_spec.supported_types = [tf.float16]
tflite_model_fp16 = converter_fp16.convert()

with open('InceptionV3_model_fp16.tflite', 'wb') as f:
    f.write(tflite_model_fp16)

# ---------------------
# 3. Full Integer Quantization (INT8)
# ---------------------

# Define representative dataset function
def representative_dataset():
    for _ in range(100):
        # Replace with a batch of input data from your dataset
        data = np.random.rand(1, *model.input_shape[1:]).astype(np.float32)
        yield [data]

converter_int8 = tf.lite.TFLiteConverter.from_keras_model(model)
converter_int8.optimizations = [tf.lite.Optimize.DEFAULT]
converter_int8.representative_dataset = representative_dataset
converter_int8.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_int8.inference_input_type = tf.uint8
converter_int8.inference_output_type = tf.uint8

tflite_model_int8 = converter_int8.convert()

with open('InceptionV3_model_int8.tflite', 'wb') as f:
    f.write(tflite_model_int8)

Mounted at /content/drive
Saved artifact at '/tmp/tmplmadll0x'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  132535016976848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132535016979536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132535016980304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132535016980112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132535016978384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132535016978576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132535002023504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132535002022928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132535002023696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132535002022160: TensorSpec(shape=(), dtype=tf.resource, na

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [ ]:
from pathlib import Path
import shutil, glob

dest = Path("/content/drive/MyDrive/DiabeticProject/tflite")
dest.mkdir(parents=True, exist_ok=True)

for f in glob.glob("*.tflite"):
    shutil.copy(f, dest / Path(f).name)

print("Copied to:", dest)

Copied to: /content/drive/MyDrive/DiabeticProject/tflite


In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
import os, time

# Where to search (adjust if you use a different folder)
base = Path('/content/drive/MyDrive')

# Find common Keras/TensorFlow save formats
candidates = []
for pattern in ['**/*.keras', '**/*.h5', '**/saved_model.pb', '**/*.index']:  # .index is TF checkpoint
    candidates += list(base.glob(pattern))

# Show top matches sorted by modified time
def pretty(p):
    try:
        mtime = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(p.stat().st_mtime))
        size = p.stat().st_size if p.is_file() else 0
    except Exception:
        mtime, size = "?", 0
    return f"{'[DIR]' if p.is_dir() else '     '}\t{mtime}\t{size:>10}\t{p}"

for p in sorted(candidates, key=lambda x: x.stat().st_mtime if x.exists() else 0, reverse=True)[:50]:
    print(pretty(p))


Mounted at /content/drive
     	2025-08-23 14:35:23	 278154169	/content/drive/MyDrive/DiabeticProject/saved_models/InceptionV3_single_split.keras
     	2025-08-23 14:34:21	 278154169	/content/drive/MyDrive/DiabeticProject/saved_models/InceptionV3_single/latest_model.keras
     	2025-08-20 17:26:57	  38367273	/content/drive/MyDrive/DiabeticProject/saved_models/MobileNetV2_single_split.keras
     	2025-08-20 17:24:10	  38367273	/content/drive/MyDrive/DiabeticProject/saved_models/MobileNetV2_single/latest_model.keras
     	2025-08-08 02:58:15	  82521738	/content/drive/MyDrive/DiabeticProject/saved_models/EfficientNetV2B0_single_split.keras
     	2025-08-08 02:55:30	  82521738	/content/drive/MyDrive/DiabeticProject/saved_models/EfficientNetV2B0_single/latest_model.keras
